In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

weather = pd.read_csv('data/weather.csv')
pallet_history = pd.read_csv('data/Pallet_history_Gold_Spike.csv')
inbound = pd.read_csv('data/inbound_loads.csv')
outbound = pd.read_csv('data/outbound_laods.csv')
demand = pd.read_csv('data/demand_kWtrain_val.csv')

In [67]:
# Merge the data frames based on the appropriate keys
# For example, to merge inbound and pallet_history data frames:

inbound_pallet = pd.merge(inbound, pallet_history, on=['Unnamed: 0'])

# Merge all the data frames into one
merged_data = pd.merge(demand, inbound_pallet, left_on=['datetime_local'], right_on=['truck_signin_datetime'])
merged_data = pd.merge(merged_data, outbound, on='truck_signin_datetime')
# merged_data = pd.merge(merged_data, weather, left_on='datetime_local', right_on='datetime')

cols_to_use = weather.columns.difference(merged_data.columns)
merged_data = pd.merge(merged_data, weather[cols_to_use], left_index=True, right_index=True, how='outer')

print(merged_data.head())
merged_data['datetime'] = merged_data['datetime'].apply(lambda x: x.value)


# Split the data into training and test sets
X = merged_data.drop('demand_kW', axis=1)
y = merged_data['demand_kW']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up the pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the performance of the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)


   Unnamed: 0_x datetime_local  demand_kW  Unnamed: 0_y  \
0           NaN            NaN        NaN           NaN   
1           NaN            NaN        NaN           NaN   
2           NaN            NaN        NaN           NaN   
3           NaN            NaN        NaN           NaN   
4           NaN            NaN        NaN           NaN   

   warehouse_order_number_x  customer_code_x load_reference_number_x  \
0                       NaN              NaN                     NaN   
1                       NaN              NaN                     NaN   
2                       NaN              NaN                     NaN   
3                       NaN              NaN                     NaN   
4                       NaN              NaN                     NaN   

  carrier_code_x  front_temperature  middle_temperature  ...  \
0            NaN                NaN                 NaN  ...   
1            NaN                NaN                 NaN  ...   
2            NaN    

AttributeError: 'str' object has no attribute 'value'